This notebook is a continuation of the features created in the other notebook.

In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

## Connect to Spotify API

In [3]:
# Setup Spotify API
# Setup Spotipy
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util


with open('credentials.json') as f:
    data = json.load(f)
    CLIENT_ID = data['SPOTIPY_CLIENT_ID']
    CLIENT_SECRET = data['SPOTIPY_CLIENT_SECRET']

In [8]:
USERNAME = 'tedman390' # spotify username
REDIRECT_URI = 'http://localhost:8000' # developer account "http://localhost:8000"
SCOPES = ['playlist-read-collaborative', 'playlist-read-private', 'user-library-read']

In [9]:
token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPES, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

In [10]:
sp = spotipy.Spotify(auth=token)

In [19]:
# Get playlist names 
# Wanted to turn into function so can use current user or input a user id, but Spotify API does not allow for accessing 
# playlists that are not belonging to the authorized user for safety.
id_name = {}
for i in sp.current_user_playlists()['items']:
        
        id_name[i['name']] = i['uri'].split(':')[2]

In [20]:
id_name

{'White people singing out of time for the ✨aesthetic✨': '1RZQLrDbFGMyjhLJEng4KK',
 'My recommendation playlist': '3GVAejf2EZ6NHCgh6kps00',
 'Doing chores and I need some bangers to keep me sane': '7mOJgSbUtIuEw1zwHR2uYf',
 'Alarm': '5TwtCQwDNOinV86biOPO8F',
 'Crip Walk': '5d6o8QqAaVN0QHSTNz9J9Y',
 "Ted Binns's top tracks ": '2H63hKSx2dJ3U7LRxKJlhZ',
 'Cavs is a being of untamable power': '0m4Q9BWl0Xu3X2J0nC99tm',
 'Songs where the funny vibrating strings sound bussin': '0yUE89luwSRaht7lSubsKb',
 'Big Sadge': '6IE8rwVcc46ekltyDlRCEA',
 'Scene Queens': '1WzuOQZBXqx1IqKpmIXiBF',
 'Roadtrip at my desk': '2dq8DRNhD87B9w5Yijpbpq',
 'Your Top Songs 2022': '37i9dQZF1F0sijgNaJdgit',
 'A playlist to study until you master every field of Physics': '63GbykWEdrZoOkwf3aIrsz',
 'The Curse of Timeland': '4FeW6jckfpgKtnz1CGGUh9',
 'Night City': '6k6Vx8iWglAz3fvvK89IvB',
 'Witcher study playlist': '6WoGeeJEvKFng25kI1Sbas',
 'System.../...Meltown': '6pgfpMaTMZXoFPEYVdzc2N',
 "You're Getting Cynical": '5

## Import spotify dataset

In [9]:
spotify_df = pd.read_csv('tracks-3.csv')

In [10]:
spotify_df['genres_upd_v1'] = spotify_df['genre_list'].apply(lambda x: re.findall(r"'([^']*)'", x))
spotify_df['genres_upd_v2'] = spotify_df['genre_list'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['genres_upd'] = np.where(spotify_df['genres_upd_v1'].apply(lambda x: not x), spotify_df['genres_upd_v2'], spotify_df['genres_upd_v1'] )

## Import complete_feature_set.csv

In [11]:
complete_feature_set = pd.read_csv('complete_feature_set.csv')

In [12]:
complete_feature_set.head()

genre|150  genre|21st  genre|432hz  genre|48g  genre|a3  genre|abc  \
0        0.0         0.0          0.0        0.0       0.0        0.0   
1        0.0         0.0          0.0        0.0       0.0        0.0   
2        0.0         0.0          0.0        0.0       0.0        0.0   
3        0.0         0.0          0.0        0.0       0.0        0.0   
4        0.0         0.0          0.0        0.0       0.0        0.0   

   genre|abstract  genre|acadienne  genre|accordeon  genre|accordion  \
0             0.0              0.0              0.0              0.0   
1             0.0              0.0              0.0              0.0   
2             0.0              0.0              0.0              0.0   
3             0.0              0.0              0.0              0.0   
4             0.0              0.0              0.0              0.0   

   genre|aceh  genre|acid  genre|acidcore  genre|acousmatic  genre|acoustic  \
0         0.0         0.0             0.0               0.0             0.0   
1         0.0         0.0             0.0               0.0             0.0   
2         0.0         0.0             0.0               0.0             0.0   
3         0.0         0.0             0.0               0.0             0.0   
4         0.0         0.0             0.0               0.0             0.0   

   genre|action  genre|adoracao  genre|adoracion  genre|adult  \
0           0.0             0.0              0.0          0.0   
1           0.0             0.0              0.0          0.0   
2           0.0             0.0              0.0          0.0   
3           0.0             0.0              0.0          0.0   
4           0.0             0.0              0.0          0.0   

   genre|adventista  genre|advocacy  genre|afghan  genre|africaine  \
0               0.0             0.0           0.0              0.0   
1               0.0             0.0           0.0              0.0   
2               0.0             0.0           0.0              0.0   
3               0.0             0.0           0.0              0.0   
4               0.0             0.0           0.0              0.0   

   genre|african  genre|afrikaans  genre|afro  genre|afrobeat  \
0            0.0              0.0         0.0             0.0   
1            0.0              0.0         0.0             0.0   
2            0.0              0.0         0.0             0.0   
3            0.0              0.0         0.0             0.0   
4            0.0              0.0         0.0             0.0   

   genre|afrocolombiano  genre|afrofuturism  genre|afrofuturismo  \
0                   0.0                 0.0                  0.0   
1                   0.0                 0.0                  0.0   
2                   0.0                 0.0                  0.0   
3                   0.0                 0.0                  0.0   
4                   0.0                 0.0                  0.0   

   genre|afroperuana  genre|afropop  genre|afroswing  genre|age  genre|aggro  \
0                0.0            0.0              0.0        0.0          0.0   
1                0.0            0.0              0.0        0.0          0.0   
2                0.0            0.0              0.0        0.0          0.0   
3                0.0            0.0              0.0        0.0          0.0   
4                0.0            0.0              0.0        0.0          0.0   

   genre|aggrotech  genre|ahead  genre|ai  genre|aikatsu  genre|ainu  \
0              0.0          0.0       0.0            0.0         0.0   
1              0.0          0.0       0.0            0.0         0.0   
2              0.0          0.0       0.0            0.0         0.0   
3              0.0          0.0       0.0            0.0         0.0   
4              0.0          0.0       0.0            0.0         0.0   

   genre|alabama  genre|alagoana  genre|alagoano  genre|alaska  \
0            0.0             0.0             0.0           0.0  

## Useful functions for visualising playlists

In [13]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Get songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): dataframe
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE DATASET
    """
    
    # Generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [14]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """ 
    Get cover art of the songs in the inputted dataframe

    Parameters: 
        df (pandas dataframe): playlist Dataframe
    """
    
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(int(len(temp) / columns + 1), columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

## My Linkin Park playlist

In [15]:
#playlist_linkin = create_necessary_outputs('Linkin Park', id_name, complete_feature_set)

In [16]:
#playlist_linkin

In [17]:
#visualize_songs(playlist_linkin)

## Creating a playlist vector

Want to be able to summarize a user's playlist in a single vector. We'll do this by summing each column, and potentially adding a recency bias so the recommender is more likely to predict new music.

In [18]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [19]:
#complete_feature_set_playlist_vector_linkin, complete_feature_set_nonplaylist_linkin = generate_playlist_feature(complete_feature_set, playlist_linkin, 1.09)

In [20]:
#complete_feature_set_playlist_vector_linkin.shape

## Create recommendations

To find similar songs, I'll use cosine similarity, by comparing the playlist vector against the features of other tracks in the database.

In [21]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [22]:
#linkin_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_linkin, complete_feature_set_nonplaylist_linkin)
#linkin_top40[['name', 'artists']]

In [23]:
#visualize_songs(linkin_top40)

## Turn into easy-to-use function

In [24]:
def get_recos_file(playlist_name, recency_bias, spotify_df, id_name, complete_feature_set):
    '''
    Exports the top 40 as a text file with track name and artist name(s) for a given user and their playlist
    
    Parameters:
        spotify_df (pandas df): spotify dataframe
        id_name (dict): dictionary mapping playlist name to playlist id
        complete_feature_set (pandas df): full dataframe with all processed ohe features
        playlist_name (str): name of the playlist to base recommendations off of
        recency_bias (float): desired recency bias (i.e. 1.09)
    '''
    # Get df of playlist tracks that are in the db
    playlist = create_necessary_outputs(playlist_name, id_name, complete_feature_set)
    
    # Create a playlist vector
    complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist, recency_bias)
    
    # Genereate top40 recommendations
    top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
    
    # Export
    name_artist = top40[['name', 'artists']]
    name_artist.to_csv(playlist_name + ' recommendations.csv', sep=",")

In [25]:
#To see available playlists from the user:
#id_name

#To use:
#get_recos_file(Name of playlist to gather recommendations from,
#               Recency bias,
#               spotify_df,
#               id_name,
#               complete_feature_set)

In [26]:
id_name

{'White people singing out of time for the ✨aesthetic✨': '1RZQLrDbFGMyjhLJEng4KK',
 'Doing chores and I need some bangers to keep me sane': '7mOJgSbUtIuEw1zwHR2uYf',
 'Alarm': '5TwtCQwDNOinV86biOPO8F',
 'Crip Walk': '5d6o8QqAaVN0QHSTNz9J9Y',
 "Ted Binns's top tracks ": '2H63hKSx2dJ3U7LRxKJlhZ',
 'Cavs is a being of untamable power': '0m4Q9BWl0Xu3X2J0nC99tm',
 'Songs where the funny vibrating strings sound bussin': '0yUE89luwSRaht7lSubsKb',
 'Big Sadge': '6IE8rwVcc46ekltyDlRCEA',
 'Scene Queens': '1WzuOQZBXqx1IqKpmIXiBF',
 'Roadtrip at my desk': '2dq8DRNhD87B9w5Yijpbpq',
 'The Curse of Timeland': '4FeW6jckfpgKtnz1CGGUh9',
 'Night City': '6k6Vx8iWglAz3fvvK89IvB',
 'System.../...Meltown': '6pgfpMaTMZXoFPEYVdzc2N',
 'YOOOORKSHIRE': '4QqJO6R5s4mstxf0QpDll1',
 'OOOOH ENTER SHIKARI': '3jBfkTG7mZYLbolkpCw8O3',
 'Psyched In': '0GfKXlXSHUgka6Tikt4CLA',
 'Chill-a-gator': '4pB8SLO3xBFNttNPFML3cJ',
 'Psyched Out': '1dSFjPwygeqeSs2Xtyhs8i',
 'Countdown-D-Day-Tezeta': '5QpdyGMQW0iE8PqM5z2uks',
 'K.G.

In [27]:
get_recos_file('Maroon V', 1.09, spotify_df, id_name, complete_feature_set)

C:\Users\edwar\AppData\Local\Temp\ipykernel_35640\2672743570.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


## Remarks

The model works! Though there are some points I want to mention.
- Personally, I like listening to music I could sing along too, which means I'm innately biased towards English music. I'd like some way of **filtering to English songs**, but that could be difficult as I'd have to find this feature for every single track in the database.
- My favourite band, King Gizzard and the Lizard Wizard has no entri in the database. Infact, there doesn't seem to be much neo-psychedelic music at all. I could fix this in the future by **combining multiple datasets, perhaps creating one of my own with a wide variety of neo-psych music**, but this will run into another issue.
- The database is massive. I already had problems working on Deepnote, with the free machine running out of RAM. I transferred back over to Jupyter notebooks on my home PC which was better, but still had issues flag up when creating the .csv files of my `complete_feature_set`. What I *should* do is **use SQL for my database, potentially though MySQL** since it is free. That next step would facilitate expanding the database.
- With that in mind, I would like to be able to do more with the Spotify API - instead of ignoring songs in my playlist that are not in the database, I **want the program to get the information for those songs**. This should be possible as all the data comes from the Spotify API, either from audio features or from artist genres. After getting this data, the program could then **add the new findings to the database** so that with each use, it gains a wider breadth of potential data.